In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn import naive_bayes
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

Read SMS messages from dataset

In [ ]:
df=pd.read_csv('../input/spam.csv', encoding='latin-1')

It contain 5572 observations and 5 columns

In [ ]:
df.shape

In [ ]:
# It shows you the top 5 rows of data
df.head()

Countplot show the counts of observations in each categorical bin(ham and spam) using bars.

In [ ]:
sns.countplot(x='v1',data=df)

Remove columns which are not signigicant.

In [ ]:
df.drop(df.iloc[:, 2:5], inplace=True, axis=1)
df.head()

Remove stopwords that don't have any significance.

In [ ]:
#It will get the stopwords used.
stopset = set(stopwords.words('english'))


TFIDF Vectorizer - Convert a collection of raw documents to a matrix of TF-IDF features. All stopwords are removed.

TF-IDF stands for “term frequency-inverse document frequency”, meaning the weight assigned to each token not only depends on its frequency in a document but also how recurrent that term is in the entire corpora. TF-IDF says how important that word is to that document with respect to the corpus.

In [ ]:
vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=stopset)

Learn vocabulary and idf from training set.

In [ ]:
vectorizer.fit(df)

Convert the categorical value to numerical value using Label Encoder

Label encoding is pretty much intuitive and straight-forward and may give you a good performance from your learning algorithm.

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
a=le.fit_transform(df['v1'])
print(list(le.classes_))
print(a)
print(list(le.inverse_transform([0, 1])))

In [ ]:
type= pd.DataFrame(data=a, columns=["type"])
df1 = pd.concat([df,type],axis=1)
print(df1.head())
df1.drop('v1',axis=1,inplace=True)
df1.head()

In [ ]:
#Dependent variable
Y=df1.type

In [ ]:
# Convert a collection of raw documents to a matrix of TF-IDF features.
# Converting text present in all sms into features.
#5572--> total number of SMSs 8578--> total no of words across all SMSs excluding the stopwords
X = vectorizer.fit_transform(df.v2)
X.shape

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
IDF(t) = log_e(Total number of documents / Number of documents with term t in it).
tf-idf score=TF(t)*IDF(t)

Let us consider the first SMS

In [ ]:
#Spliting the SMS to separate the text into individual words
SMS_1= df1['v2'][0].split()
print(SMS_1)

In [ ]:
#Number of words in SMS_1 are 20 but only 14 tf-idf values are generated as remaining were stopwords which are excluded
print(len(SMS_1))
print(X[0])


In [ ]:
## Most frequent word appearing in the SMS
max(SMS_1)

In [ ]:
#TF IDF values for all words 
print(X)

In [ ]:
#To get the word in a particular position
vectorizer.get_feature_names()[7826]

Word at position 7826 is 'true'

In [ ]:
#Split and train the model 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

In [ ]:
#Train Naive Bayes Classifier
NB = naive_bayes.MultinomialNB()
model=NB.fit(X_train, Y_train)
Y_predict=model.predict(X_test)

In [ ]:
print(X_test, Y_predict)

In [ ]:
print(Y_test)

In [ ]:
Y_predict1=pd.DataFrame(Y_predict)
print(Y_predict1)

In [ ]:
print(Y_test.shape,Y_predict.shape)

Probability of assigning a SMS to a specific class

In [ ]:
prob=model.predict_proba(X_test)
prob

In [ ]:
model.predict_proba(X_test)[:,1]

In [ ]:
#Model's accuracy
roc_auc_score(Y_test, model.predict_proba(X_test)[:,1])